## Consumo API feriados
Conexión sencilla a API de Feriados del Gobierno, transformación de datos mediante librería pandas y carga en bigquery

In [11]:
# Importación librerías
import requests
import pandas as pd
from datetime import datetime
import pytz
from google.cloud import bigquery

In [2]:
# Variables globales para conversión de 
# Fechas a zona horaria Chile
fmt= '%Y-%m-%d'
cl = pytz.timezone("America/Santiago")
now_dt = datetime.now(cl)

In [3]:
# response.json()

In [4]:

def response_to_dataframe(response):
    """
    Función que da forma a la data recibida 
        Parámetro: response API
        Returns: dataframe
    """

    df_feriados = pd.DataFrame(response.json())

    if df_feriados.empty:
        return df_feriados
    
    df_feriados["irrenunciable"] = df_feriados["irrenunciable"].astype(int).astype(bool)
    df_feriados["descripcion"] = df_feriados["nombre"]
    df_feriados["feriado"] = True
    return df_feriados[["fecha", "feriado", "irrenunciable", "descripcion"]]

In [5]:
# df_feriados=response_to_dataframe(response)
# df_feriados
# df_feriados.info()

In [6]:
def gen_rango_fechas():
    """
    Función retorna dataframe de fechas, cuyas filas van desde el 01 de agosto de 2019 
    al último día del año de la consulta
    """
    
    now_d = now_dt.date().replace(month=12, day=31)
    ultimo_diaannio = now_d.strftime(fmt)

    rango_fechas = pd.date_range('2019-08-01',  ultimo_diaannio, freq= 'D').astype(str)
    df_rango_fechas = pd.DataFrame(rango_fechas)
    df_rango_fechas.rename(columns={0 : 'fecha'}, inplace=True)
    return df_rango_fechas

In [7]:
def gen_df_final(df_feriados, df_rango_fechas):
    """
    Función une ambos dataframes 
    """
    df_final= df_feriados.join(df_rango_fechas.set_index('fecha'), how= 'right', on= 'fecha')
    df_final.reset_index(inplace=True)    
    df_final["feriado"].fillna(False, inplace=True)

    df_final= df_final[['fecha', 'feriado', 'irrenunciable', 'descripcion']]

    return df_final

#df_final.iloc[-10:-1]

In [8]:
def upload_data_bq(df_final):
    """
    Función define esquema de la tabla en BQ y sobreescribe datos
    """
    _google_key = "../../Credenciales/servacc_bigquery.json"

    bq_table = "conexion-datos-rdf.diccionarios.dicc_feriados"
    
    client = bigquery.Client.from_service_account_json(_google_key)

    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField(
                "fecha", bigquery.enums.SqlTypeNames.STRING),
            bigquery.SchemaField(
                "feriado", bigquery.enums.SqlTypeNames.BOOLEAN),
            bigquery.SchemaField(
                "irrenunciable", bigquery.enums.SqlTypeNames.BOOLEAN),
            bigquery.SchemaField(
                "descripcion", bigquery.enums.SqlTypeNames.STRING),
        ],
        write_disposition="WRITE_TRUNCATE",
        clustering_fields=["fecha", "feriado"],
        
    )

    job = client.load_table_from_dataframe(df_final, bq_table, job_config=job_config)

    print(job.result())  # Wait for the job to complete.

In [9]:
def etl_dicc_feriados():
    """
    Función extrae, transforma y carga la data
    """
    user_agent = {'User-agent': 'Mozilla/5.0'}
    endpoint= "https://apis.digital.gob.cl/fl/feriados"
    response = requests.get(endpoint, headers= user_agent)
    
    df_feriados = response_to_dataframe(response)
    df_rango_fechas = gen_rango_fechas()
    df_final = gen_df_final(df_feriados, df_rango_fechas)
    upload_data_bq(df_final)
    
    return df_final

In [ ]:
#Llamado a la función etl
df_final= etl_dicc_feriados()
df_final